In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class mLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, embed_size, output_size):
        super(mLSTM, self).__init__()

        self.hidden_size = hidden_size
        # input embedding
        self.encoder = nn.Embedding(input_size, embed_size)
        # lstm weights
        self.weight_fm = nn.Linear(hidden_size, hidden_size)
        self.weight_im = nn.Linear(hidden_size, hidden_size)
        self.weight_cm = nn.Linear(hidden_size, hidden_size)
        self.weight_om = nn.Linear(hidden_size, hidden_size)
        self.weight_fx = nn.Linear(embed_size, hidden_size)
        self.weight_ix = nn.Linear(embed_size, hidden_size)
        self.weight_cx = nn.Linear(embed_size, hidden_size)
        self.weight_ox = nn.Linear(embed_size, hidden_size)
        # multiplicative weights
        self.weight_mh = nn.Linear(hidden_size, hidden_size)
        self.weight_mx = nn.Linear(embed_size, hidden_size)
        # decoder
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, inp, h_0, c_0):
        # encode the input characters
        inp = self.encoder(inp)
        # calculate the multiplicative matrix
        m_t = self.weight_mx(inp) * self.weight_mh(h_0)
        # forget, input and output gates
        f_g = torch.sigmoid(self.weight_fx(inp) + self.weight_fm(m_t))
        i_g = torch.sigmoid(self.weight_ix(inp) + self.weight_im(m_t))
        o_g = torch.sigmoid(self.weight_ox(inp) + self.weight_om(m_t))
        # intermediate cell state
        c_tilda = torch.tanh(self.weight_cx(inp) + self.weight_cm(m_t))
        # current cell state
        cx = f_g * c_0 + i_g * c_tilda
        # hidden state
        hx = o_g * torch.tanh(cx)

        out = self.decoder(hx.view(1,-1))

        return out, hx, cx

    def init_hidden(self):
        h_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        c_0 = Variable(torch.zeros(1, self.hidden_size)).cuda()
        return h_0, c_0



In [2]:
# Converte string para uma lista de inteiros
from unidecode import unidecode
import random

def generate_hidden(text, rnn):
    hidden, cell = rnn.init_hidden()
    text_tensor = char_tensor(text).cuda()
    for p in range(len(text)):
        output, hidden, cell = rnn(text_tensor[p], hidden, cell)
    return hidden, cell

def char_tensor(string):
    string = unidecode(string)
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = ord(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor('The omega (Ω) symbol\n'))  

tensor([ 84, 104, 101,  32, 111, 109, 101, 103,  97,  32,  40,  79,  41,  32,
        115, 121, 109,  98, 111, 108,  10])


In [3]:
embed_size = 128 # ascii representation
hidden_size = 2048
rnn = mLSTM(embed_size, hidden_size, embed_size, embed_size).cuda()
rnn.load_state_dict(torch.load("lstm_11.pth"))
rnn.eval()

hidden, cell = generate_hidden("Adorei! Chegou bem cedo!", rnn)


In [4]:
hidden, hidden.size()

(tensor([[ 1.7533e-01, -1.9787e-03,  5.5819e-03,  ...,  5.8692e-01,
           8.6428e-01,  3.4329e-03]], device='cuda:0', grad_fn=<ThMulBackward>),
 torch.Size([1, 2048]))

In [5]:
cell, cell.size()

(tensor([[ 1.8169e-01, -1.7234e-02,  1.3710e-02,  ...,  9.7225e-01,
           1.3564e+00,  9.8336e-03]], device='cuda:0', grad_fn=<ThAddBackward>),
 torch.Size([1, 2048]))

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = mLSTM(embed_size, hidden_size, embed_size, embed_size).cuda()
        self.rnn.load_state_dict(torch.load("lstm_11.pth"))
        self.fc1 = nn.Linear(2*hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, int(hidden_size/4))
        self.fc3 = nn.Linear(int(hidden_size/4), int(hidden_size/2))
        self.fc4 = nn.Linear(int(hidden_size/2), 5)
        self.dropout = nn.Dropout(0.5) 
        
    def forward(self, text_tensor):
#         hidden, cell = rnn.init_hidden()
#         for p in range(len(text_tensor)):
#             _, hidden, cell = rnn(text_tensor[p], hidden, cell)
        
        with torch.no_grad():
            hidden, cell = rnn.init_hidden()
            for p in range(len(text_tensor)):
                _, hidden, cell = rnn(text_tensor[p], hidden, cell)
        
        
        x = torch.cat((hidden, cell), 0).view(1, -1)
        x = F.relu(self.dropout(self.fc1(x)))
        x = F.relu(self.dropout(self.fc2(x)))
        x = F.relu(self.dropout(self.fc3(x)))
        x = F.relu(self.fc4(x))
        return x

In [7]:
import pandas as pd
df_train = pd.read_csv("../data/interim/train.csv")
df_test = pd.read_csv("../data/interim/test.csv")

In [8]:
train_data = []
for score, message in zip(df_train.score.values, df_train.message.values):
    train_data.append([score, message])
    
test_data = dict()
for score, message in zip(df_test.score.values, df_test.message.values):
    try:
        test_data[score].append([score, message])
    except:
        test_data[score] = [[score, message]]

In [9]:
net = Net().cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
loss_metric = nn.CrossEntropyLoss()

In [10]:
for epoch in range(10):
    epoch_loss = 0
    random.shuffle(train_data)
    for i, (score, message) in enumerate(train_data):
        net.zero_grad()
        pred = net(char_tensor(message).cuda())
        score = torch.LongTensor([score-1]).cuda()
        loss = loss_metric(pred, score)
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        
        if i % 1000 == 0 and i != 0:
            print("Epoch: {}, it: {}, Loss: {}".format(epoch, i, loss.data.item() / i))

Epoch: 0, it: 1000, Loss: 0.0001454589366912842
Epoch: 0, it: 2000, Loss: 0.0008495268225669861
Epoch: 0, it: 3000, Loss: 0.0005689544677734375
Epoch: 0, it: 4000, Loss: 0.0016373497247695922
Epoch: 0, it: 5000, Loss: 0.00015884501934051513
Epoch: 0, it: 6000, Loss: 0.0003354875246683757
Epoch: 0, it: 7000, Loss: 0.0007261536461966379
Epoch: 0, it: 8000, Loss: 0.0002571975886821747
Epoch: 0, it: 9000, Loss: 0.00017786706818474662
Epoch: 0, it: 10000, Loss: 7.931135892868042e-05
Epoch: 0, it: 11000, Loss: 9.168766845356334e-05
Epoch: 0, it: 12000, Loss: 0.00019247700770696005
Epoch: 0, it: 13000, Loss: 8.23208001943735e-06
Epoch: 0, it: 14000, Loss: 7.237553596496582e-05
Epoch: 0, it: 15000, Loss: 0.00015099992752075196
Epoch: 0, it: 16000, Loss: 1.200959086418152e-05
Epoch: 0, it: 17000, Loss: 5.536518377416274e-05
Epoch: 0, it: 18000, Loss: 0.00023754294713338215
Epoch: 0, it: 19000, Loss: 0.00017676718611466256
Epoch: 0, it: 20000, Loss: 2.6400935649871825e-05
Epoch: 0, it: 21000, Lo

KeyboardInterrupt: 